<a href="https://colab.research.google.com/github/anandadda/AIH_T113/blob/master/model_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
ls

drive/  sample_data/


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import tensorflow.keras as keras

In [0]:
x=np.load('/content/drive/My Drive/CDAC/x_dat.npy')

In [0]:
y=np.load('/content/drive/My Drive/CDAC/y_dat.npy')

In [0]:
x.shape

(20065, 224, 224, 3)

In [0]:
y.shape

(20065,)

In [0]:
num_classes=y.max()+1

In [0]:
import random

data=list(zip(x,y))
random.shuffle(data)

X_dat=np.array([i for i,j in data])
Y_dat=np.array([j for i,j in data])

In [0]:
ny=np.zeros((Y_dat.shape[0],num_classes))

for i in range(Y_dat.shape[0]):
  ny[i,Y_dat[i]]=1

In [0]:
X_train=X_dat[:18000]
Y_train=ny[:18000]

X_test=X_dat[18000:]
Y_test=ny[18000:]

In [0]:
model= keras.models.load_model('/content/drive/My Drive/CDAC/weights.h5')

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
base_input (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        base_input[0][0]                 
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
____________________________________________________________________________________________

In [0]:
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input

Using TensorFlow backend.


In [0]:
import sklearn as sk

In [0]:
temp_mod=keras.models.Model(inputs=model.inputs,outputs=model.layers[-3].output)

In [0]:
temp_mod.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
base_input (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        base_input[0][0]                 
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
______________________________________________________________________________________________

In [0]:
for layer in temp_mod.layers:
  layer.trainable = False

In [0]:
temp_mod.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
base_input (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        base_input[0][0]                 
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
______________________________________________________________________________________________

In [0]:
model=keras.models.Sequential()

In [0]:
model.add(temp_mod)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 2048)              23561152  
Total params: 23,561,152
Trainable params: 0
Non-trainable params: 23,561,152
_________________________________________________________________


In [0]:
model.add(keras.layers.Dense(256,activation='relu'))

W0901 08:38:52.612251 140152075777920 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model.add(keras.layers.Dropout(0.75))

W0901 08:38:52.661959 140152075777920 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [0]:
model.add(keras.layers.Dense(num_classes,activation='softmax'))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 2048)              23561152  
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 85)                21845     
Total params: 24,107,541
Trainable params: 546,389
Non-trainable params: 23,561,152
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
model.fit(X_train,Y_train,batch_size=32,epochs=10,validation_split=0.1)

W0901 08:38:53.759477 140152075777920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 16200 samples, validate on 1800 samples
Epoch 1/10
16200/16200 [==============================] - 141s 9ms/sample - loss: 0.2433 - acc: 0.9882 - val_loss: 0.2411 - val_acc: 0.9884
Epoch 2/10
16200/16200 [==============================] - 135s 8ms/sample - loss: 0.2410 - acc: 0.9884 - val_loss: 0.2402 - val_acc: 0.9884
Epoch 3/10
16200/16200 [==============================] - 135s 8ms/sample - loss: 0.2405 - acc: 0.9884 - val_loss: 0.2388 - val_acc: 0.9885
Epoch 4/10
16200/16200 [==============================] - 135s 8ms/sample - loss: 0.2399 - acc: 0.9884 - val_loss: 0.2378 - val_acc: 0.9886
Epoch 5/10
16200/16200 [==============================] - 135s 8ms/sample - loss: 0.2393 - acc: 0.9884 - val_loss: 0.2358 - val_acc: 0.9887
Epoch 6/10
16200/16200 [==============================] - 135s 8ms/sample - loss: 0.2391 - acc: 0.9884 - val_loss: 0.2368 - val_acc: 0.9886
Epoch 7/10
16200/16200 [==============================] - 135s 8ms/sample - loss: 0.2390 - acc: 0.9884 - val_lo

In [0]:
model.evaluate(X_test,Y_test)

2065/2065 [==============================] - 16s 8ms/sample - loss: 0.2364 - acc: 0.9887


[0.23638263641344723, 0.9886795]

In [0]:
ls

 20180402-114759.zip  'Colab Notebooks'/   pecan/   state_detection/
 CDAC/                 Docs/               raw/     vgg_modified.h5


In [0]:
model.save('vgg_modified.h5')